In [1]:
%pylab inline
import altair as alt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../91-DIVOC/pages/covid-visualization/jhu-data.csv",index_col=0)

In [4]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# Keep only data from the top 10 countries with the highest total number of Confirmed cases
top10countries = list(df.groupby("Country_Region").sum()["Confirmed"].nlargest(10).index)
df = df[df["Country_Region"].isin(top10countries)]

In [6]:
selectedRows = df[(df["Confirmed"]==0)&(df["Recovered"]==0)&(df["Active"]==0)&(df["Deaths"]==0)].index
df = df.drop(selectedRows, axis=0)

In [7]:
df = df[df["Confirmed"]>100]

In [9]:
df[df["Country_Region"]=="United States"]

,Country_Region,Province_State,Confirmed,Recovered,Active,Deaths,Date
81,United States,NaN,108.0,8.0,89.0,11.0,03-04-2020
85,United States,NaN,176.0,8.0,156.0,12.0,03-05-2020
96,United States,NaN,372.0,8.0,347.0,17.0,03-07-2020
48,United States,California,144.0,2.0,140.0,2.0,03-10-2020
77,United States,New York,173.0,0.0,173.0,0.0,03-10-2020
...,...,...,...,...,...,...,...
114,United States,Texas,758.0,0.0,0.0,9.0,03-23-2020
118,United States,Virginia,254.0,0.0,0.0,6.0,03-23-2020
119,United States,Washington,2221.0,0.0,0.0,109.0,03-23-2020
121,United States,Wisconsin,425.0,0.0,0.0,5.0,03-23-2020


In [10]:
chart = alt.Chart(df).mark_line().encode(
    x = alt.X("Date"),
    y= alt.Y("Confirmed",aggregate="sum"),
    color = alt.Color("Country_Region")
)
chart.interactive()

alt.Chart(...)